In [5]:
#%esptool erase --port=COM5 --baud=460800
#%esptool --port=COM5 --baud=460800 esp32 "C:\Users\Admin\Documents\GitHub\3-axis-controller\esp32\esp32-2022618-v1.19.1.bin"

esptool not found on path
esptool not found on path


In [1]:
%serialconnect to --port COM5 --baud=115200

Connecting to --port=COM5 --baud=115200 
MicroPython v1.19.1 on 2022-06-18; ESP32 module with ESP32
Type "help()" for more information.
>>>[reboot detected 0]repl is in normal command mode
[\r\x03\x03] b'\r\n>>> '
[\r\x01] b'\r\n>>> \r\nraw REPL; CTRL-B to exit\r\n>' Ready.


In [2]:
from machine import Pin
from time import sleep

In [3]:
# PIN ASSIGNMENT FUNCTIONS

# function that assigns rotary encoders X, Y and Z channels (REC) to pins
def pins_REC(pinA, pinB):
    outPinA = Pin(pinA, Pin.IN)
    outPinB = Pin(pinB, Pin.IN)
    return outPinA, outPinB

# functions that assigns motor to pins
def pins_motor(pIN1, pIN2, pIN3, pIN4):
    IN1 = Pin(pIN1, Pin.OUT)
    IN2 = Pin(pIN2, Pin.OUT)
    IN3 = Pin(pIN3, Pin.OUT)
    IN4 = Pin(pIN4, Pin.OUT)
    pins = [IN1, IN2, IN3, IN4]
    return pins

In [4]:
# MOTOR FUNCTIONS

# function that assigns the step sequence
def step_sequence(n):
    # clockwise sequence
    if n > 0:
        print('clockwise sequence')
        sequence = [[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]]  
    # counterclockwise sequence
    elif n < 0:
        print('counterclockwise sequence')
        sequence = [[0,0,0,1],[0,0,1,0],[0,1,0,0],[1,0,0,0]]
    # freezing sequence
    else:
        sequence = [[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,0]]
        print('freezing sequence')
    return(sequence)
        
# function that runs n steps on motor x
def run_step(n, motor_x):
    sequence = step_sequence(n)
    pins = motor_x
    for k in range (0, abs(n)):
        for step in sequence:
            for i in range(len(pins)):
                pins[i].value(step[i])
                sleep(0.001)
    return

# function that assigns motor rotations depending on translation axis
def mov(vector):
    # clockwise
    if vector[1]=="+":
        if vector[0] == "x":
            run_step(-7, motor_a)
            run_step(7, motor_b)
        elif vector[0] == "y":
            run_step(4, motor_c)
            run_step(4, motor_b)
            run_step(-8, motor_c)
        elif vector[0] == "z":
            run_step(8, motor_a)
            run_step(8, motor_b)
            run_step(8, motor_c)
    # counterclockwise
    elif vector[1]=="-":
        if vector[0] == "x":
            run_step(7, motor_a)
            run_step(-7, motor_b)
        elif vector[0] == "y":
            run_step(-4, motor_a)
            run_step(-4, motor_b)
            run_step(8, motor_c)
        elif vector[0] == "z":
            run_step(-8, motor_a)
            run_step(-8, motor_b)
            run_step(-8, motor_c)
    return       

In [5]:
# ROTARY ENCODER FUNCTION

# function that translates REC signals into motor rotations
def REC_translation(axis, outPinA, outPinB):
    oldA = outPinA.value() 
    oldB = outPinB.value()
    sleep(0.005)

    vector = axis  
    A = outPinA.value()
    B = outPinB.value()    
    A_state = A-oldA
    B_state = B-oldB
       
        # channel A / clockwise
    if  A_state == 1 and B==0:
        vector += "+"
        mov(vector)
        # channel B / counterclockwise
    elif B_state == 1 and A==0:
        vector += "-"
        mov(vector)
    else:
        sleep(0.001)
     
    oldA = A
    oldB = B
    print('rec translation')
    
    return

In [6]:
# PROGRAM

# assign pins to the 3 rotary encoders channels (REC)
xA,xB = pins_REC(15,2)
yA,yB = pins_REC(16,17)
zA,zB = pins_REC(5,18)

# assign pins to the 3 motors
motor_a = pins_motor(26, 25, 14, 27)
motor_b = pins_motor(22, 23, 21, 19)
motor_c = pins_motor(1, 3, 9, 10)

# rotary encoders translation
while True:
    REC_translation("x",xA,xB)
    REC_translation("y",yA,yB)
    REC_translation("z",zA,zB)
    sleep(0.001)


 ...                                                                                                                                                                                                                                   .                                                                                                                              ..............................                                                                                                                                                                                 .                                                                                                                                                                              .................                                                                                                                                            .
**[ys] <class 'serial.serialutil.SerialException'>
**[ys] ClearCommError failed (PermissionError(13, 